In [30]:
import pandas as pd
from matplotlib import pyplot as plt
import os
from IPython.core.interactiveshell import InteractiveShell

In [31]:
path = "../output/kmeans/"
df_each = []
for filename in os.listdir(path):
    f = os.path.join(path, filename)
    if os.path.isfile(f):
        temp = pd.read_csv(f)
        temp = temp[temp.name.apply(lambda x: x != "name")]
        df_each.append((temp, filename))

In [32]:
print(len(df_each))
df_each[0][0].columns

36


Index(['name', 'run_num', 'params', 'reduction_time', 'train_time', 'accuracy',
       'original_std_sum', 'original_std_mean', 'original_std_median',
       'original_std_max', 'original_std_min', 'original_sparsity',
       'original_shape', 'transformed_std_sum', 'transformed_std_mean',
       'transformed_std_median', 'transformed_std_max', 'transformed_std_min',
       'transformed_sparsity', 'transformed_shape', 'characteristics'],
      dtype='object')

In [37]:
chosen_df = df_each[20][0]
grouped = chosen_df.groupby(by=["name", "params", "original_shape"])
min_ind = grouped["reduction_time"].idxmin()
chosen_df.loc[min_ind].groupby(by=["name", "params", "original_shape"], group_keys=True).apply(lambda x: x)[["accuracy", "reduction_time"]]

accuracy  \
name                params                   original_shape                 
JL transform        {'ep': 0.05, 'de': 0.05} (1000, 1000)   16       0.95   
                                             (1000, 10000)  42       1.00   
                                             (1000, 20000)  68       1.00   
                                             (10000, 1000)  94       1.00   
                                             (10000, 10000) 120      1.00   
...                                                                   ...   
sparse JL transform {'ep': 0.9, 'de': 0.1}   (10000, 1000)  93       1.00   
                                             (10000, 10000) 119      1.00   
                                             (10000, 20000) 145      0.98   
                                             (20000, 1000)  171      1.00   
                                             (20000, 10000) 197      1.00   

                                                                 reduction_time  
name                params                   original_shape                      
JL transform        {'ep': 0.05, 'de': 0.05} (1000, 1000)   16         0.026032  
                                             (1000, 10000)  42         0.601206  
                                             (1000, 20000)  68         2.896686  
                                             (10000, 1000)  94         0.044360  
                                             (10000, 10000) 120        2.799881  
...                                                                         ...  
sparse JL transform {'ep': 0.9, 'de': 0.1}   (10000, 1000)  93         0.045460  
                                             (10000, 10000) 119        0.600290  
                                             (10000, 20000) 145        1.188127  
                                             (20000, 1000)  171        0.076571  
                                             (20000, 10000) 197        0.938322  

[208 rows x 2 columns]

In [11]:
def sol_2(n):
    cnt = 0
    while n > 1:
        if n == 3:
            cnt += 2
            break
        elif n % 2 == 0:
            n /= 2
        elif n % 4 == 1:
            n -= 1
        else:
            n += 1
        cnt += 1
    return cnt

[(i, sol_2(i)) for i in range(2, 100)]

[(2, 1),
 (3, 2),
 (4, 2),
 (5, 3),
 (6, 3),
 (7, 4),
 (8, 3),
 (9, 4),
 (10, 4),
 (11, 5),
 (12, 4),
 (13, 5),
 (14, 5),
 (15, 5),
 (16, 4),
 (17, 5),
 (18, 5),
 (19, 6),
 (20, 5),
 (21, 6),
 (22, 6),
 (23, 6),
 (24, 5),
 (25, 6),
 (26, 6),
 (27, 7),
 (28, 6),
 (29, 7),
 (30, 6),
 (31, 6),
 (32, 5),
 (33, 6),
 (34, 6),
 (35, 7),
 (36, 6),
 (37, 7),
 (38, 7),
 (39, 7),
 (40, 6),
 (41, 7),
 (42, 7),
 (43, 8),
 (44, 7),
 (45, 8),
 (46, 7),
 (47, 7),
 (48, 6),
 (49, 7),
 (50, 7),
 (51, 8),
 (52, 7),
 (53, 8),
 (54, 8),
 (55, 8),
 (56, 7),
 (57, 8),
 (58, 8),
 (59, 8),
 (60, 7),
 (61, 8),
 (62, 7),
 (63, 7),
 (64, 6),
 (65, 7),
 (66, 7),
 (67, 8),
 (68, 7),
 (69, 8),
 (70, 8),
 (71, 8),
 (72, 7),
 (73, 8),
 (74, 8),
 (75, 9),
 (76, 8),
 (77, 9),
 (78, 8),
 (79, 8),
 (80, 7),
 (81, 8),
 (82, 8),
 (83, 9),
 (84, 8),
 (85, 9),
 (86, 9),
 (87, 9),
 (88, 8),
 (89, 9),
 (90, 9),
 (91, 9),
 (92, 8),
 (93, 9),
 (94, 8),
 (95, 8),
 (96, 7),
 (97, 8),
 (98, 8),
 (99, 9)]

In [20]:
sol_2(10000000000)

40

In [23]:
mp = {}

def minops(n, steps):
    if n == 1: return steps
    if n in mp: return mp[n] + steps
    if n % 2 == 0:
        cnt = minops(n/2, steps + 1)
        mp[n] = cnt - steps
        return cnt
    cnt = min(minops(n - 1, steps + 1), minops(n + 1, steps + 1))
    mp[n] = cnt - steps
    return cnt

minops(10000000000, 0)

40